In [45]:
import warnings
warnings.filterwarnings("ignore")

import sys
sys.path.append("/home/shoinoue/Git/CosyVoice/")
sys.path.append("/home/shoinoue/Git/CosyVoice/third_party/Matcha-TTS/")

from cosyvoice.cli.cosyvoice import CosyVoice
from cosyvoice.utils.file_utils import load_wav
# import torchaudio
import torch
from scipy.io.wavfile import write as write_wav

import glob
import os
import numpy as np
from tqdm import tqdm

import IPython
def play_audio(data, rate):
    IPython.display.display(IPython.display.Audio(data=data,rate=rate))
    
cosyvoice = CosyVoice('/mntcephfs/data/audiow/shoinoue/Model/models/cosyvoice/CosyVoice-300M')

2024-07-30 17:44:30.673531493 [E:onnxruntime:Default, provider_bridge_ort.cc:1480 TryGetProviderInfo_CUDA] /onnxruntime_src/onnxruntime/core/session/provider_bridge_ort.cc:1193 onnxruntime::Provider& onnxruntime::ProviderLibrary::Get() [ONNXRuntimeError] : 1 : FAIL : Failed to load library libonnxruntime_providers_cuda.so with error: libcurand.so.10: cannot open shared object file: No such file or directory

2024-07-30 17:44:30.673565069 [W:onnxruntime:Default, onnxruntime_pybind_state.cc:743 CreateExecutionProviderInstance] Failed to create CUDAExecutionProvider. Please reference https://onnxruntime.ai/docs/execution-providers/CUDA-ExecutionProvider.html#requirements to ensure all dependencies are met.
2024-07-30 17:44:31,313 WETEXT INFO found existing fst: /mntcephfs/lab_data/shoinoue/miniconda3/envs/cosyvoice/lib/python3.8/site-packages/tn/zh_tn_tagger.fst
2024-07-30 17:44:31,313 WETEXT INFO found existing fst: /mntcephfs/lab_data/shoinoue/miniconda3/envs/cosyvoice/lib/python3.8/sit

- Cross Lingual for L2-ARCTIC

In [38]:
# speakers = ["ASI", "RRBI", "SVBI", "TNI", "ABA", "SKA", "YBAA", "ZHAA"]
speakers = ["ASI", "RRBI", "SVBI", "TNI"]

In [51]:
sample_num = 100000000
repeat_num = 2
save = True
play = True

for spk in speakers:
    print(spk)
    base_dir = f"/mntcephfs/lab_data/shoinoue/Dataset/L2-ARCTIC/{spk}/wav/"
    text_dir = "./SPAT/transliteration/"
    save_dir = f"/mntcephfs/lab_data/shoinoue/Dataset/CosyVoice/{spk}_allsamples/English/wav/"
    os.makedirs(save_dir, exist_ok=True)
    audiofiles = glob.glob(base_dir + "*.wav")
    audiofiles.sort()
    
    for path in tqdm(audiofiles[:sample_num]):
        prompt_speech_16k = load_wav(path, 16000)
        basename = os.path.basename(path)[:-4]
        textpath = text_dir + basename + ".npy"
        sentences = np.load(textpath, allow_pickle=True).item()
        for r in range(repeat_num):
            output = cosyvoice.inference_cross_lingual(f'<|en|>{sentences["English"]}', prompt_speech_16k)
            savepath = save_dir + basename + f"-{r}.wav"
            if save:
                # torchaudio.save(savepath, output['tts_speech'], 22050)
                write_wav(savepath, 22050, output["tts_speech"][0].numpy())
            if play:
                x = load_wav(savepath, 16000)
                play_audio(prompt_speech_16k, 16000)
                play_audio(x, 16000)

ASI


  0%|          | 0/1131 [00:00<?, ?it/s]

  0%|          | 1/1131 [00:12<3:50:33, 12.24s/it]

  0%|          | 1/1131 [00:21<6:50:58, 21.82s/it]


KeyboardInterrupt: 

In [49]:
output["tts_speech"][0].numpy()

array([-9.5686427e-04,  8.3655301e-05, -4.4412972e-04, ...,
       -1.0770484e-04, -6.6602515e-05,  3.0067089e-04], dtype=float32)